In [133]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from PIL import Image
import glob
import helper
import argparse
import sys
import csv

In [134]:
def read_metric(out_file,stage,name_file, name,name_model,fold_out,fold_in,epochs):
    loss_file = open(("predictions/{}/pred_loss_{}{}_{}_foldout{}_foldin{}_{}epochs.txt").format(out_file,stage,name_file,name_model,fold_out,fold_in,epochs))
    filedata = loss_file.read()
    filedata = filedata.replace("bce",",bce")
    filedata = filedata.split(",")
    metric=[]
    for i in filedata:
        i = i.strip(" ")
        if str(i).startswith(name):
            i = i.split(" ")
            metric.append(float(i[1]))
            
    plt.close('all')
    f = plt.figure()
    y_axe = np.asarray(metric)
    
    x =np.asarray(list(range(0,len(metric))))
    plt.xlabel(("Number of {} images").format(stage))
    plt.ylabel(name)
    plt.title(("predictions_with_{}").format(name) )
    plt.plot(x,y_axe,label = name_file)
    plt.show()
    f.savefig(("predictions/{}/metric_{}_{}{}_{}_foldout{}_foldin{}_{}epochs.pdf").format(out_file,name,stage,name_file,name_model,fold_out,fold_in,epochs), bbox_inches='tight')
    print('Ctd:',len(x),name,np.mean(metric))
    plt.close()

    #return metric


In [135]:
def values_metric(filedata,name_metric):
    m_metric = []
    for i in filedata:
        i = i.strip(" ")
        if str(i).startswith(name_metric):
            i = i.split(" ")
            m_metric.append(float(i[1]))
    return m_metric

In [136]:
def plot_history_train(out_file,name_file,name_model,fold_out,fold_in,epochs):
    file = open(("history/{}/history_model{}_{}_foldout{}_foldin{}_{}epochs.txt").format(out_file,name_file,name_model,fold_out,fold_in,epochs), "r")
    

        
    filedata = file.read() 
    filedata = filedata.replace("dataloader",",dataloader")
    filedata = filedata.replace("saving",",saving")
    filedata = filedata.replace("\n",", \n")



    filedata = filedata.split(",")
    loss = []
    for i in filedata:
        i = i.strip(" ")

        if str(i).startswith("loss:"):
            i = i.split(":")
            loss.append(float(i[1]))
            
    plt.close('all')
    f= plt.figure()  
    y_train = np.asarray(loss[0::2])
    y_val = np.asarray(loss[1::2])

    x =np.asarray(list(range(0,len(y_val))))
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("loss graph")
    plt.plot(x[1:120:],y_train[1:120:],label = 'train_loss') 
    plt.plot(x[1:120:],y_val[1:120:],'k', label = 'val_loss')
    plt.legend()
    plt.show()
    f.savefig(("history/{}/loss_convergence{}_{}_foldout{}_foldin{}_{}epochs.pdf").format(out_file,name_file,name_model,fold_out,fold_in,epochs), bbox_inches='tight') 
    plt.close()
 #  

In [137]:
def percentage_dice(stage='test',name_file='_VHR_60_fake',out_file='VHR',name_model='UNet11',fold_out=0,fold_in=0,epochs=40, count=30): # #HR •dist
    
    num_patches_test= 227
    
    loss_file = open(("predictions/{}/pred_loss_{}{}_{}_foldout{}_foldin{}_{}epochs.txt").format(out_file,stage,name_file,name_model,fold_out,fold_in,epochs))
  
    filedata = loss_file.read()
    filedata = filedata.replace("bce",",bce")
    filedata = filedata.split(",")

    m_jaccard=values_metric(filedata,"jaccard")
    m_dice=values_metric(filedata,"dice")
    
    val_file = (("predictions/{}/inputs_{}{}_{}_foldout{}_foldin{}_{}epochs_{}.npy").format(out_file, stage, name_file,name_model,fold_out,fold_in,epochs, count))
    pred_file =(("predictions/{}/pred_{}{}_{}_foldout{}_foldin{}_{}epochs_{}.npy").format(out_file, stage, name_file,name_model,fold_out,fold_in,epochs, count))
    label_file = (("predictions/{}/labels_{}{}_{}_foldout{}_foldin{}_{}epochs_{}.npy").format(out_file, stage, name_file,name_model,fold_out,fold_in,epochs, count))

    val_images = np.load(val_file)
    #pred_images = np.load(pred_file)
    val_label = np.load(label_file)
    print('val_pred')
    print(val_label.shape)
    #input_images_rgb = [helper.reverse_transform(x,out_file) for x in val_images[:num_patches_test,0,:3,:,:]]   #new metrics
    # Map each channel (i.e. class) to each color
    #target_masks_rgb = [helper.masks_to_colorimg(x) for x in val_label[:num_patches_test,0,:3,:,:]]
    #pred_rgb = [helper.masks_to_colorimg(x) for x in pred_images[:num_patches_test,0,:,:,:]]
    
      
    #print('input_shape')
    #print(len(input_images_rgb))
    
    #print('target_shape')
    #print(len(target_masks_rgb))
  
    #print('pred_rgb')
    #print(len(pred_rgb))
    
    
    percentage = []
        
    #with open('percentage_file.csv', mode='w') as percentage_file:
    #    percentage_writer = csv.writer(percentage_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    #    percentage_writer.writerow(perc)

    for i in range(0, num_patches_test):
        
        #print("\nPatch number is {}".format(i))
        #plt.figure()    
        #plt.imshow(input_images_rgb[i])
        #plt.figure()
        #plt.imshow(target_masks_rgb[i])
        #plt.savefig('patch.jpg')

        #mask_count= np.sum(val_label[i])
        #print("Mask count is {}".format(mask_count))
        #non_mask_count= np.count_nonzero(val_label[i])
        #print("Mask non_count is {}".format(non_mask_count))
        perc = np.sum(val_label[i])/(512*512)*100
        #print("Percentage is {}".format(perc))
        
        
        percentage.append([i, m_dice[i],m_jaccard[i],perc])
        
    np.savetxt("percentageUNET_foldout0_foldin4_epochs_100.csv", percentage, delimiter=",")  
        
    #percentage_file.close()
    
    #####esto colocaria ##### ##### ##### ##### ##### ##### #####
    # F: es que el rgb ya tiene colores y hay valores alrededor que se ven como negro, pero son valores pequeños
    #mask = target_masks_rgb[4]
    #mask=mask.transpose(1, 2, 0).reshape(mask.shape[1],-1)
    #mask=(mask > 0).astype(np.uint8) 
    #equals0=(mask==1).astype(np.uint8)        
    #sum_percent=np.sum(equals0)/(512*512)*100
    #print(sum_percent)
    ##### ##### ##### ##### ##### ##### ##### ##### ##### #####
    
    #plt.figure()
    #plt.imshow(pred_rgb[4])
    
    #name_output=("{}{}_{}_foldout{}_foldin{}_{}epochs").format(stage, name_file,name_model,fold_out,fold_in,epochs)
  
    # stage + name_file + name_model+'_foldin' +str(fold_in)
    # helper.plot_side_by_side([input_images_rgb, target_masks_rgb, pred_rgb],filedata, out_file, name_output, save=1)


In [138]:
def main():   
    #parser = argparse.ArgumentParser()
    #arg = parser.add_argument
    #arg('--out-file', type=str, default='160', help='For example 160 or 512')
    #arg('--stage', type=str, default='test', help='For example test or val')
    #arg('--name-file', type=str, default='_8_percent', help='For example _6_percent')
    #arg('--name-model', type=str, default='UNet11', choices=['UNet11','UNet','AlbuNet34','SegNet'])
    #arg('--fold-out', type=int, help='fold train test', default=0)
    #arg('--fold-in', type=int, help='fold train val', default=0)  
    #arg('--n-epochs', type=int, help='epochs in which the model was trained', default=40)  
    #arg('--count', type=int, help='number of img to plot', default=94)        
    
    #args = parser.parse_args()
    

    #plot_history_train(args.out_file, args.name_file,args.name_model,args.fold_out,args.fold_in,args.n_epochs)   
    percentage_dice(stage='test',name_file='_100_percent_512',out_file='512',name_model='UNet',fold_out=0,fold_in=4,epochs=100, count=227) 
#     python plotting.py --out-file '512' --stage 'test' --name-file '_100_percent_512' --name-model 'UNet11' --count 227 --n-epochs  70 --fold-in $j


if __name__ == '__main__':
    main()


val_pred
(227, 1, 1, 512, 512)
